In [1]:
import tensorboard.backend.event_processing.event_accumulator as ec

files = [
"/home/sontung/thesis/location-based-generative/logs/20200725-100335/events.out.tfevents.1595664215.ipvs8gtx.24823.0",
"/home/sontung/thesis/location-based-generative/logs/20200726-041935/events.out.tfevents.1595729975.ipvs8gtx.222051.0",
    "/home/sontung/thesis/location-based-generative/logs/sim20200725-114336/events.out.tfevents.1595670216.ipvs8gtx.32638.0",
    "/home/sontung/thesis/location-based-generative/logs/sim20200726-052721/events.out.tfevents.1595734041.ipvs8gtx.224681.0"
]

eas = []
for f in files:
    ea = ec.EventAccumulator(f)
    ea.Reload()
    eas.append(ea)

print(eas[0].Tags())

/home/sontung/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sontung/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sontung/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sontung/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

{'images': [], 'audio': [], 'histograms': [], 'scalars': ['train/loss', 'train/iou', 'val/loss', 'val/acc', 'val/iou'], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}


In [4]:
data2 = [[du.value for du in ea.Scalars("train/iou")] for ea in eas]
# data2 = [[du.value for du in ea.Scalars("train/loss")] for ea in eas]
print(len(data2))
print([i[-1] for i in data2])

4
[0.7814750075340271, 0.8274691700935364, 0.8054813146591187, 0.8292793035507202]


In [18]:
import numpy as np
import json
import sys
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

from scipy import signal


plotly.io.orca.config.executable = '/home/sontung/tools/orca-1.3.1.AppImage'


fig = make_subplots(rows=1, cols=2, subplot_titles=("on PBW data", "on sim data"), shared_yaxes=True)

names = ["with optim", "w/o optim", "only kld"]
colors = [dict(color='firebrick'), dict(color='rgb(255,0,0)')]


fig.add_trace(
    go.Scatter(
        x=list(range(len(data2[0]))),
        y=signal.savgol_filter(data2[0],53, 3),
        mode="lines",
        name=names[0],
        line=dict(color='rgb(0,68,27)'),
        showlegend=True
    ), row=1, col=1,
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(data2[0]))),
        y=signal.savgol_filter(data2[1],53, 3),
        mode="lines",
        name=names[1],
        line=dict(color='rgb(127,39,4)'),
        showlegend=True
    ), row=1, col=1,
)

##

fig.add_trace(
    go.Scatter(
        x=list(range(len(data2[0]))),
        y=signal.savgol_filter(data2[2],53, 3),
        mode="lines",
        name=names[0],
        line=dict(color='rgb(0,68,27)'),
        showlegend=False
    ), row=1, col=2,
)


fig.add_trace(
    go.Scatter(
        x=list(range(len(data2[0]))),
        y=signal.savgol_filter(data2[3],53, 3),
        mode="lines",
        name=names[1],
        line=dict(color='rgb(127,39,4)'),
        showlegend=False
    ), row=1, col=2
)

fig.update_xaxes(title_text="iteration", row=1, col=1)
fig.update_xaxes(title_text="iteration", row=1, col=2)

fig.update_xaxes(ticks="outside")
fig.update_yaxes(ticks="outside", col=1)

fig.update_layout(
    yaxis_title="IoU",
    legend=dict(x=0.0, y=-0.25),

    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#000000"
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=80, r=10, t=30, b=80)

#     plot_bgcolor='rgba(0,0,0,0)'
)
fig.update_layout(legend_orientation="h")
# fig.update_layout( yaxis_type="log")

fig.show()
fig.write_image("learning.png", width=1000, height=None, scale=2)
